# 4.5 在transfomer模块中连接注意力层和线性层

在本节中，我们将实现transfomer模块，这是GPT和其他大型语言模型（LLM）架构的基本构建块。
该模块在 1.24 亿参数的 GPT-2 架构中重复了十几次，结合了我们之前介绍过的几个概念：多头注意力、层归一化、dropout、前馈层和 GELU 激活函数，如图 4.13 所示。
在下一节中，我们将把这个ransfomer模块连接到 GPT 架构的其余部分。

**图 4.13 transfomer模块的图示。
图的底部显示了输入的token，这些标记已经被嵌入到768维的向量中。
每一行对应一个token的向量表示。
transfomer模块的输出是与输入具有相同维度的向量，然后可以将其馈送到 LLM 中的后续层。**

![fig4.13](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-13.jpg?raw=true)

如图 4.13 所示，transfomer模块结合了多个组件，包括第 3 章中的屏蔽多头注意力模块和我们在第 4.3 节中实现的前馈模块。

当transfomer模块处理输入序列时，序列中的每个元素（例如，单词或子词标记）都由固定大小的向量表示（在图 4.13 的情况下，为 768 维）。
transfomer模块内部的操作，包括多头注意力和前馈层，都旨在以保持其维度的方式转换这些向量。

多头注意力模块中的自注意力机制的思想是，它能够识别并分析输入序列中各元素之间的关系。
与此同时，前馈网络在每个位置独立地修改数据。
这种组合不仅可以更细致地理解和处理输入，还可以增强模型处理复杂数据模式的整体能力。

在下面代码中，我们可以按照如下方法创建 Transformer模块：

### 代码示例4.6 GPT 的transfomer模块组件

In [1]:
import torch
import torch.nn as nn
from torch.nn import LayerNorm

In [2]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])
        
    def forward(self, x):
        #A
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_resid(x)
        x = x + shortcut # Add the original input back
        
        shortcut = x #B
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut #C
        return x

上述给出的代码在 PyTorch 中定义了一个 TransformerBlock 类，包括一个多头注意力机制（MultiHeadAttention）和一个前馈网络（FeedForward），这两者都是根据提供的配置字典（例如 GPT_CONFIG_124M）来配置的。

在这两个组件之前应用层归一化 (LayerNorm)，并在它们之后应用 dropout，以正则化模型并防止过度拟合。
这也称为 Pre-LayerNorm。
在较旧的架构中，例如原始的transfomer模块，层归一化是在自注意力和前馈网络之后应用的，被称为 Post-LayerNorm，这通常会导致较差的训练动态。

该类还实现了前向传递，其中每个组件后面都有一个快捷连接，该连接将块的输入添加到其输出。
一关键特性有助于训练过程中梯度在网络中的流动，并如4.4节所解释的那样，改善了深度模型的学习效果。

使用我们之前定义的 GPT_CONFIG_124M 字典，让我们实例化一个transfomer模块并为其提供一些示例数据。

In [3]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.linear1 = nn.Linear(cfg["emb_dim"], cfg["emb_dim"] * 4)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(cfg["emb_dim"] * 4, cfg["emb_dim"])
        self.dropout = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x
    
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads #A
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) #B
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
        'mask',
         torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x) #C
        queries = self.W_query(x) #C
        values = self.W_value(x) #C
        
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) #D
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) #D
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)#D

        keys = keys.transpose(1, 2) #E
        queries = queries.transpose(1, 2) #E
        values = values.transpose(1, 2) #E

        attn_scores = queries @ keys.transpose(2, 3) #F
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] #G

        attn_scores.masked_fill_(mask_bool, -torch.inf) #H

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) #I
        #J
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) #K
        return context_vec

GPT_CONFIG_124M = {
    "vocab_size": 50257, # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768, # Embedding dimension
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 0.1, # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

In [4]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768) #A
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


输出如下所示:

Input shape: torch.Size([2, 4, 768]) \
Output shape: torch.Size([2, 4, 768])

从代码输出中我们可以看到，transfomer模块在其输出中保持了输入的维度，这表明transfomer架构在整个网络中处理数据序列时不改变其形状。

在整个transfomer模块架构中保持形状并不是偶然的，而是其设计的一个关键方面。
这种设计使其能够有效应用于广泛的序列到序列任务，其中每个输出向量直接对应于输入向量，从而保持一对一的关系。
然而，正如我们在第 3 章中学到的那样，输出是一个上下文向量，它封装了整个输入序列的信息。
这意味着，虽然序列的物理维度（长度和特征大小）在通过transfomer模块时保持不变，但每个输出向量的内容被重新编码以整合来自整个输入序列的上下文信息。

在本节中实现的transfomer模块让我们拥有了所有构建块，正如图4.14所示，这些构建块是在下一节中实现GPT架构所需要的。

**图 4.14 展示了迄今为止我们在本章中实现的不同概念的心智模型。**

![fig4.14](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-14.jpg?raw=true)

如图 4.14 所示，transfomer模块结合了层归一化、前馈网络（包括 GELU 激活）和快捷连接，我们在本章早些时候已经介绍过。
如我们将在即将到来的章节中看到的，这个transfomer模块将构成我们将要实现的GPT架构的主要组成部分。